# Inception-V3 图像分类

Inception-V3模型是谷歌在大型图像数据库ImageNet 上训练好了一个图像分类模型，这个模型可以对1000种类别的图片进行图像分类。但现成的Inception-V3无法对“花” 类别图片做进一步细分，因此本实验的花朵识别实验是在Inception-V3模型基础上采用迁移学习方式完成对“花” 类别图片进一步细分的实验。

Inception v3模型大约有2500万个参数，分类一张图像就用了50亿的乘加指令。在一台没有GPU的现代PC上，分类一张图像转眼就能完成。

In [10]:
# 导入依赖
import os.path
import re
import matplotlib.pyplot as plt
#import tensorflow as tf
import tensorflow._api.v2.compat.v1 as tf
import numpy as np
import InceptionV3Model    # inception模型

In [11]:
# 检查tf版本号
tf.__version__

'2.3.1'

In [12]:
model_dir = r"./ImageNet/"   # 模型目录
image = r"./image/panda.jpg"

In [13]:
#将类别ID转换为人类易读的标签
class NodeLookup(object):
  def __init__(self,
               label_lookup_path=None,
               uid_lookup_path=None):
    if not label_lookup_path:
      label_lookup_path = os.path.join(
          model_dir, 'imagenet_2012_challenge_label_map_proto.pbtxt')
    if not uid_lookup_path:
      uid_lookup_path = os.path.join(
          model_dir, 'imagenet_synset_to_human_label_map.txt')
    self.node_lookup = self.load(label_lookup_path, uid_lookup_path)

  def load(self, label_lookup_path, uid_lookup_path):
    if not tf.gfile.Exists(uid_lookup_path):
      tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not tf.gfile.Exists(label_lookup_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = tf.gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
      parsed_items = p.findall(line)
      uid = parsed_items[0]
      human_string = parsed_items[2]
      uid_to_human[uid] = human_string

    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = tf.gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
      if line.startswith('  target_class:'):
        target_class = int(line.split(': ')[1])
      if line.startswith('  target_class_string:'):
        target_class_string = line.split(': ')[1]
        node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
      if val not in uid_to_human:
        tf.logging.fatal('Failed to locate: %s', val)
      name = uid_to_human[val]
      node_id_to_name[key] = name

    return node_id_to_name

  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]

In [14]:
# 读取训练好的Inception-v3模型来创建graph
def create_graph():   # 在当前的版本中，gfile已经定义在io包的file_io.py中
    with tf.gfile.FastGFile(os.path.join(
        model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

In [15]:
# 读取图片
image_data = tf.gfile.FastGFile(image, 'rb').read()

Instructions for updating:
Use tf.gfile.GFile.


In [16]:
# 创建graph
create_graph()

In [17]:
sess = tf.Session()

#Inception-v3模型的最后一层softmax的输出
softmax_tensor= sess.graph.get_tensor_by_name('softmax:0')

#输入图像数据，得到softmax概率值（一个shape=(1,1008)的向量）
predictions = sess.run(softmax_tensor,{'DecodeJpeg/contents:0': image_data})
#(1,1008)->(1008,)
predictions = np.squeeze(predictions)

# ID --> English string label.
node_lookup = NodeLookup()

#取出前5个概率最大的值（top-5)
top_5 = predictions.argsort()[-5:][::-1]
for node_id in top_5:
  human_string = node_lookup.id_to_string(node_id)
  score = predictions[node_id]
  print('%s (score = %.5f)' % (human_string, score))

sess.close()

giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca (score = 0.92052)
teddy, teddy bear (score = 0.00169)
lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens (score = 0.00122)
earthstar (score = 0.00076)
forklift (score = 0.00055)
